In [ ]:
from tensorflow import keras
from keras import layers
import numpy as np

In [ ]:
data = np.load('../../data/othello_train_data_model_enhanced.npz' ,allow_pickle=True)
X, y = data['X'], data['y']
X = np.where(X == 3, 0, X)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train.reshape(-1, 64)).reshape(-1, 8, 8)
X_val = scaler.transform(X_val.reshape(-1, 64)).reshape(-1, 8, 8)

In [ ]:
print("Training data shape:", X_train.shape, y_train.shape)
print("-" *10)

print(X_train[50])
print(f'This game winner was {y_train[50]}')

In [ ]:
network = keras.Sequential([
    layers.Input(shape=(8,8,1)),

    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    
    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Flatten(),
    layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.ReLU(),


    layers.Dense(1, activation='tanh')
    ])
network.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='mse',metrics=['r2_score'])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_r2_score',     
    factor=0.5,              
    patience=5,              
    min_lr=1e-5,             
    verbose=1
)


history = network.fit(X_train, y_train, epochs=200, batch_size=32,validation_split=0.2,callbacks=[early_stop,lr_scheduler])

In [ ]:
network.evaluate(X_val, y_val)

In [ ]:
#Uncomment below to save the  model
#network.save("../../models/model_v3.keras")

In [ ]:
data = np.load('../../data/othello_train_data_model_enhanced.npz' ,allow_pickle=True)
X, y = data['X'], data['y']
X = np.where(X == 3, 0, X)
print(X.shape, y.shape)
for i,_ in enumerate(X):
    print(X[i])
    print(f'Label: {y[i]}')

In [ ]:
model = keras.models.load_model('../../models/model_v3.keras')
print(model.predict(X_val[:10]))
print(y_val[:10])